In [ ]:
import joblib
import click
import json
import time
import os
import itertools
import collections.abc
from shutil import copy,SameFileError
from glob import glob

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pennylane as qml
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
# from qiskit_ibm_provider import IBMProvider
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="Attempted to compute the gradient of a tape with no trainable parameters.")

from qregress.quantum.Quantum import QuantumRegressor
from qregress.quantum.Evaluate import evaluate
from qregress.scripts.settings import ANSATZ_LIST, ENCODER_LIST
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import InconsistentVersionWarning

In [ ]:


# device='qiskit.aer'
# device_backend = FakeCairoV2()
# backend = AerSimulator.from_backend(device_backend)
# device = qml.device(device, wires=num_qubits, backend=backend, shots=shots)


device='qulacs.simulator'
num_qubits=5
device = qml.device(device, wires=num_qubits)


In [ ]:
def circuit(features, parameters):
    #  builds the circuit with the given encoder and variational circuits.
    #  encoder and variational circuits must have only two required parameters, params/feats and wires
    for i in range(re_upload_depth):
        params = initial_parameters.reshape(re_upload_depth,-1)[i]
        
        encoder(features, wires=range(num_qubits))
        variational(params, wires=range(num_qubits))

    return qml.expval(qml.PauliZ(0))

In [ ]:
def get_depth(enc,var,rud,features,parameters):
    circuit_gates=0
    circuit_depth=0
    for i in range(rud):
        params = initial_parameters.reshape(rud,-1)[i]
        
        encoder_specs=qml.specs(qml.QNode(encoder,device),level=3)(features, wires=range(num_qubits))['resources']
        var_specs=qml.specs(qml.QNode(variational,device),level=3)(params, wires=range(num_qubits))['resources']

        circuit_gates+=var_specs.num_gates + encoder_specs.num_gates
        circuit_depth+=var_specs.depth + encoder_specs.depth
    return circuit_gates,circuit_depth

In [ ]:
with open(os.path.join('../BSE-calc-test/5qubit/IQP_Full-CRX/2_model_log.csv'),'r') as f:
    # Create df
    timedf = pd.DataFrame(pd.to_datetime([i.split(',')[0] for i in f.readlines() if '[' in i], format='%a %b %d %H:%M:%S %Y'),columns=['Time'])
    # Get time between iterations and get the mean time
    timedf['Iter time']=[i.total_seconds() for i in timedf['Time'].diff()]

In [ ]:
# # Define the number of qubits and the input data
# num_qubits = 5
# data = np.random.rand(num_qubits)

# # Create a quantum device


# # Define the IQPEmbedding circuit
# @qml.qnode(device)
# def circuit(data):
#     qml.IQPEmbedding(features=data, wires=range(num_qubits))
#     return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]

# # Draw the circuit
# fig, ax = qml.draw_mpl(circuit, expansion_strategy='device',level=3)(data)
# plt.show()

In [ ]:
()

In [ ]:
flat_results = []
for i in glob('../BSE-calc-test/1000_5qubit/*/*results.json'):
    p = os.path.basename(os.path.dirname(i))
    
    with open(i,'r') as f:
        og_results=json.load(f)
    ENCODER, ANSATZ = p.split('_')
    # print(ENCODER, ANSATZ)
    encoder = ENCODER_LIST[ENCODER]
    variational=ANSATZ_LIST[ANSATZ]

    LAYERS=1
    re_upload_depth=1
    variational.layers = LAYERS
    variational.set_wires(range(num_qubits))
    

    num_params = variational.num_params * re_upload_depth
    generator = np.random.default_rng(12958234)
    initial_parameters = qml.numpy.array(generator.uniform(-np.pi, np.pi, num_params), requires_grad=True)
    feat = generator.uniform(-1, 1, num_qubits)
    
    qnode = qml.QNode(circuit, device)   

    # print(f"{p} AL_{LAYERS} RUD_{re_upload_depth}\n",circuit_specs)
    # qml.draw_mpl(qnode,level=3)(feat,initial_parameters)
    # plt.show()
    
    circuit_specs=qml.specs(qnode,level=3)(feat,initial_parameters)
    # if ENCODER=="IQP":
    #     print(circuit_specs['resources'].depth)
    circuit_gates,circuit_depth= get_depth(encoder,variational,1,feat,initial_parameters)
    times = []
    # print(i)
    for j in glob(os.path.join(f"../BSE-calc-test/5qubit/{p}",'*model_log.csv')):
        with open(j,'r') as f:
            # Create df
            timedf = pd.DataFrame(pd.to_datetime([i.split(',')[0] for i in f.readlines() if '[' in i], format='%a %b %d %H:%M:%S %Y'),columns=['Time'])
            # Get time between iterations and get the mean time
            timedf['Iter time']=[i.total_seconds() for i in timedf['Time'].diff()]
            times.append(timedf)
    
    flat_results.append((1,1,p,og_results['MAE_train'],og_results['MAE_test'],circuit_gates,circuit_depth,pd.concat(times)['Iter time'].mean()))

In [ ]:
df = pd.DataFrame(flat_results,columns=['AL','RUD','pairs','MAE Train','MAE Test','num_gates','depth','mean_iter'])

In [ ]:
palette=sns.color_palette('Paired',8)

In [ ]:
median_Train,median_Test=np.median(df[['MAE Train',"MAE Test"]].values,axis=0)

In [ ]:
df.sort_values(by=['depth','mean_iter'])

In [ ]:
df[df['pairs']=='M-M-CZ_HWE-CNOT']

In [ ]:
beststats = '\n'.join([f'{k}: {v:.2f}' if type(v)!=str else f'{k}: {v}' for k,v in df[df['pairs']=='M-M-CZ_HWE-CNOT'].rename(columns={'mean_iter':'Mean Iteration','pairs':'ENCODER_ANSATZ','depth':"Depth",'num_gates':'Number of Gates'}).iloc[-1].iloc[2:].to_dict().items()])

In [ ]:
killed = np.genfromtxt('kill_list.txt',dtype=str)
killeddf = pd.concat([df[df['pairs'] == i] for i in killed])

In [ ]:
SMALL_SIZE = 8
MEDIUM_SIZE = 16
BIGGER_SIZE = 12

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title

fig, ax = plt.subplots(1,2,sharey=True,figsize=(12,6))
# sns.scatterplot(df,x='MAE Train',y='mean_iter',label='Train',ax=ax[0],color=palette[0],edgecolor='k')
sns.scatterplot(df,x='MAE Test',y='mean_iter',label='Test',ax=ax[0],color=palette[1],edgecolor='k')
sns.scatterplot(df[df['pairs']=='M-M-CZ_HWE-CNOT'].rename(columns={'MAE Test':'Best Test','MAE Train':'Best Train'}).melt(value_vars=['Best Test','Best Train'],id_vars=['pairs','mean_iter']), x='value',y='mean_iter',hue='variable',palette=palette[4:6],ax=ax[0],edgecolor='k')
sns.scatterplot(killeddf,x='MAE Test',y='mean_iter',label='Test',ax=ax[0],color='orange',edgecolor='k')


ax[0].set_xlim(0,40)
# ax[0].set_ylim(0,300)
ax[0].set_ylabel('Mean Time/Training Iteration (s)')
ax[0].set_xlabel('MAE (kcal/mol)')
ax[0].legend()

ax[1].scatter(df['depth'],df['mean_iter'],color=palette[3],edgecolor='k')
ax[1].set_xlim(1,200)
ax[1].set_xlabel('Circuit Depth')
sns.scatterplot(df[df['pairs']=='M-M-CZ_HWE-CNOT'], x='depth',y='mean_iter',color=palette[5],ax=ax[1],edgecolor='k',label='M-M-CZ_HWE-CNOT')
ax[1].text(60,10,beststats,fontsize=12)
plt.tight_layout()
# plt.savefig('AllBSE5_circuitdepth_MAE_vs_iterationtime.png',dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
df[df['pairs']=='M-M-CZ_HWE-CNOT'].rename(columns={'MAE Test':'Best Test','MAE Train':'Best Train'}).melt(value_vars=['Best Test','Best Train'],id_vars=['pairs','mean_iter'])

In [ ]:
SMALL_SIZE = 8
MEDIUM_SIZE = 16
BIGGER_SIZE = 12

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title

fig, ax = plt.subplots(1,2,sharey=True,figsize=(12,6))
sns.scatterplot(df,x='MAE Train',y='mean_iter',label='Train',ax=ax[0],color=palette[0],edgecolor='k')
sns.scatterplot(df,x='MAE Test',y='mean_iter',label='Test',ax=ax[0],color=palette[1],edgecolor='k')
sns.scatterplot(df[df['pairs']=='M-M-CZ_HWE-CNOT'].rename(columns={'MAE Test':'Best Test','MAE Train':'Best Train'}).melt(value_vars=['Best Test','Best Train'],id_vars=['pairs','mean_iter']), x='value',y='mean_iter',hue='variable',palette=palette[4:6],ax=ax[0],edgecolor='k')
sns.scatterplot(killeddf,x='MAE Train',y='mean_iter',label='Test',ax=ax[0],color='orange',edgecolor='k')
# ax[0].set_xlim(15,40)
# ax[0].set_ylim(0,100)
ax[0].set_ylabel('Mean Time/Training Iteration (s)')
ax[0].set_xlabel('MAE (kcal/mol)')
ax[0].legend()

ax[1].scatter(df['depth'],df['mean_iter'],color=palette[3],edgecolor='k')
ax[1].scatter(killeddf['depth'],killeddf['mean_iter'],color='orange',edgecolor='k')
ax[1].set_xlim(1,50)
ax[1].set_xlabel('Circuit Depth')
sns.scatterplot(df[df['pairs']=='M-M-CZ_HWE-CNOT'], x='depth',y='mean_iter',color=palette[5],ax=ax[1],edgecolor='k',label='M-M-CZ_HWE-CNOT')
ax[1].text(15,70,beststats,fontsize=12)
plt.tight_layout()
# # plt.savefig('AllBSE5_circuitdepth_MAE_vs_iterationtime.png',dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
palette

In [ ]:
df[df['pairs']=='M-M-CZ_HWE-CNOT']